In [2]:
from PIL import Image
import os

LOAD_RAW_FOLDER = "crawlers/images/full/"
SAVE_4K_FOLDER = "./data/4k/"
SAVE_540_FOLDER = "./data/540/"
os.makedirs(SAVE_4K_FOLDER, exist_ok=True)
os.makedirs(SAVE_540_FOLDER, exist_ok=True)

skip_existing = True

In [3]:
for f in os.listdir(LOAD_RAW_FOLDER):
    save_path = os.path.join(SAVE_4K_FOLDER, f)
    if skip_existing and os.path.exists(save_path):
        continue

    im = Image.open(os.path.join(LOAD_RAW_FOLDER, f))
    width, height = im.size

    target_width = 3840
    target_height = 2160
    im1 = None
    im2 = None
    if width > target_width:
        left = (width - target_width) / 2
        im1 = im.crop((left, 0, width - left, height))
    elif target_width > width:
        left = (target_width - width) / 2
        im1 = Image.new(im.mode, (target_width, height), (0, 0, 0))
        im1.paste(im, (left, 0))
    else:
        im1 = im

    if height > target_height:
        top = (height - target_height) / 2
        im2 = im1.crop((0, top, target_width, height - top))
    elif target_height > height:
        top = (target_height - height) / 2
        im2 = Image.new(im1.mode, (target_width, target_height), (0, 0, 0))
        im2.paste(im1, (0, top))
    else:
        im2 = im1

    im2.save(save_path)

C:\Users\dontpanic\AppData\Local\Programs\Python\Python38\lib\site-packages\PIL\Image.py:2855: DecompressionBombWarning: Image size (144000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
C:\Users\dontpanic\AppData\Local\Programs\Python\Python38\lib\site-packages\PIL\Image.py:2855: DecompressionBombWarning: Image size (117839442 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
C:\Users\dontpanic\AppData\Local\Programs\Python\Python38\lib\site-packages\PIL\Image.py:2855: DecompressionBombWarning: Image size (97934004 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
C:\Users\dontpanic\AppData\Local\Programs\Python\Python38\lib\site-packages\PIL\Image.py:2855: DecompressionBombWarning: Image size (169382400 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
C:\Users\dontpanic\AppData\Local\

In [4]:
for f in os.listdir(SAVE_4K_FOLDER):
    save_path = os.path.join(SAVE_540_FOLDER, f)
    if skip_existing and os.path.exists(save_path):
        continue

    im = Image.open(os.path.join(SAVE_4K_FOLDER, f))
    target_size = (960, 540)
    im = im.resize(target_size)
    im.save(save_path, "PNG")